# Secp256k1

Constants from https://en.bitcoin.it/wiki/Secp256k1

In [1]:
%run "secp256k1-ecdsa.ipynb"

# Pay-to-contract

In [2]:
import hashlib

In [3]:
def p2c(m, keypair):
    (x, P) = keypair
    h = int(hashlib.sha256(str(P.xy()[0]).encode('hex') + m).hexdigest(), 16)
    cx = h + x
    CP = h * G + P
    return (cx, CP)

# Sign-to-contract

In [4]:
def s2c(m, d, c):
    e = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    k = int(N.random_element(1, N.order() - 1))
    R = k * G # (k, r) is like a one-time key pair
    r = int(R.xy()[0]) % N.order()
    assert(r % N.order() != 0)

    HRc = int(hashlib.sha256(str(r).encode('hex') + c).hexdigest(), 16)
    kk = k + HRc
    Q = R + HRc*G
    q = int(Q.xy()[0]) % N.order()
    assert(q % N.order() != 0)

    s = (e + d*q)/kk % N.order()
    return ([q,s], R)

In [5]:
def s2cVerify(sig, R, m, P, c):
    if (verify(sig, m, P) == false):
        return false
    
    r = int(R.xy()[0]) % N.order()
    HRc = int(hashlib.sha256(str(r).encode('hex') + c).hexdigest(), 16)
    T = R + HRc * G
    t = int(T.xy()[0]) % N.order()
    
    return t == sig[0]

# Example

In [6]:
(x, P) = keypair()

In [7]:
(cx, CP) = p2c(b"contract", (x, P))
p2cSig = sign(b"Hello world", cx)

verify(p2cSig, b"Hello world", CP)

True

In [8]:
(s2cSig, s2cR) = s2c(b"Hello world", x, b"contract")

verify(s2cSig, b"Hello world", P), s2cVerify(s2cSig, s2cR, b"Hello world", P, b"contract")

(True, True)